<a href="https://colab.research.google.com/github/matsunagalab/tutorial_analyzingMDdata/blob/main/01_mdtraj_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MDTraj basics

## MDTrajのインストールとデータのダウンロード

In [1]:
# Colab environment で mdtraj をインストールする
!pip install -q condacolab
import condacolab
condacolab.install_mambaforge()
!mamba install mdtraj

# T4 LysozymeのMDデータの取得
# 溶媒は削除済みの wildtype.psf と wildtype.dcd をダウンロードする。
!gdown 1Lu22Z7ARKSws77sBaYr84iCcQsnTHId9
!gdown 1Eh0SSSxgAmv7LI2NvAY43tLMLI63lSa-

✨🍰✨ Everything looks OK!

Looking for: ['mdtraj']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.10.*
  - python 3.10.*
  - python_abi 3.10.* *cp310*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  All requested packages already installed

Downloading...
From: https://drive.google.com/uc?id=1Lu22Z7ARKSws77sBaYr84iCcQsnTHId9
To: /content/wildtype.pdb
100% 214k/214k [00:00<00:00, 64.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1Eh0SSSxgAmv7LI2NvAY43tLMLI63lSa-
From (redirected): https://drive.google.com/uc?id=1Eh0SSSxgAmv7LI2NvAY43tLMLI63lSa-&confirm=t&uuid=d7da6f80-ee86-4424-aaf6-df993897b6e7
To: /content/wildtype.dcd
100% 318M/318M [00:01<00:00, 221MB/s]


## MDTrajを使ってみる

以下の文書は MDTrajのドキュメント https://www.mdtraj.org/1.9.7/examples/introduction.html を主に日本語訳したものです。

ディスクからトラジェクトリをロードして始めます。MDTrajは自動的にファイル拡張子を解析し、適切なローダーを使用します。
Topology情報を含まないファイルをロードするには、Gromacs XTCファイルのようなもの、
Topologyを記述する何か、たとえばPDBファイル、をtopキーワード引数で提供する必要があります。

In [2]:
import mdtraj as md
import numpy as np

# DCDファイルからトラジェクトリを読み込み、対応するPSFトポロジーファイルを使用する
# トラジェクトリオブジェクト t が作成される
t = md.load('/content/wildtype.dcd', top='/content/wildtype.psf')

# t の中身を出力する
print(t)

FileNotFoundError: [Errno 2] No such file or directory: '/content/wildtype.psf'

Trajectoryのフレームの一部だけに興味がある場合は、それをスライスすることができます。

In [ ]:
# 最初の10フレームを見てみましょう
print(t[1:10])

<mdtraj.Trajectory with 9 frames, 2643 atoms, 164 residues, and unitcells>


In [ ]:
# それとも最後のフレームを見る？
print(t[-1])

<mdtraj.Trajectory with 1 frames, 2643 atoms, 164 residues, and unitcells>


Trajectoryオブジェクトには多くの情報があります。自明なのは
デカルト座標です。それらはxyzの下のnumpy配列として格納されています。全ての
座標の長さはナノメートルで保存されています。時間単位
はピコ秒です。角度はdegree (radianではない) で保存されています。

Trajectoryオブジェクトが持っているプロパティやメソッドは[ドキュメント](https://www.mdtraj.org/1.9.7/api/generated/mdtraj.Trajectory.html#)で確認することができます。

In [ ]:
t.xyz.shape

(6768, 2643, 3)

In [ ]:
np.mean(t.xyz)

0.37758622

In [ ]:
# 最初の10フレームのシミュレーション時間（ピコ秒）
t.time[0:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
# 最後のフレームの単位セルの長さは？（ナノメートル）
t.unitcell_lengths[-1]

array([7.5977883, 7.5977883, 7.5977883], dtype=float32)

Trajectoryをディスクに戻すのは簡単です。

In [ ]:
# hdf5形式はファイル内にトポロジを保存するので便利です
t[::2].save('halftraj.h5')

In [ ]:
# フォーマットは拡張子に基づいて解析されます、またはフォーマット固有のsaveメソッドを呼び出すこともできます
t[0:10].save_dcd('first-ten-frames.dcd')

TrajectoryオブジェクトにはTopologyオブジェクトへが含まれており、便利に利用することが出来ます。例えば、alpha carbon のみを含むトラジェクトリのコピーを保存したい場合、以下のようにできます。

In [ ]:
atoms_to_keep = [a.index for a in t.topology.atoms if a.name == 'CA']
t.restrict_atoms(atoms_to_keep) # この操作はトラジェクトリオブジェクトを変化させます
t.save('CA-only.h5')

Topogyオブジェクトが持つプロパティやメソッドは[ドキュメント](https://www.mdtraj.org/1.9.7/api/generated/mdtraj.Topology.html)で確認することができます。